#### Imports

In [17]:
import sys
!{sys.executable} -m pip install fitparse
from fitparse import FitFile
import os
import pandas as pd

#### Getting Path of all running moves

In [14]:
directory = 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/'
running_files = []
for filename in os.listdir(directory):
    if filename.endswith('Running.fit') : 
        running_files.append(os.path.join(directory, filename))
        
running_files[:5]

['C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_02_11_46_40_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_02_12_32_23_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_19_19_49_13_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_20_19_56_41_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_24_18_17_34_Running.fit']

#### Creating DataFrame with all the data from the watch

In [70]:
# Initializing categories we want to store
ls_move_code = []
ls_record_code = []
ls_altitude = []
ls_cadence = []
ls_distance = []
ls_enhanced_altitude = []
ls_enhanced_speed = []
ls_heart_rate = []
ls_position_lat = []
ls_position_long = []
ls_speed = []
ls_timestamp = []
ls_vertical_speed = []

move_code = 1
for filepath in running_files:
    fitfile = FitFile(filepath)
    # Get all data messages that are of type record
    for record in fitfile.get_messages('record'):
        record_code = 1
        # Go through all the data entries in this record and fill the lists
        ls_move_code.append(move_code)
        ls_record_code.append(record_code)
        ls_altitude.append(record.get_value('altitude'))
        ls_cadence.append(record.get_value('cadence'))
        ls_distance.append(record.get_value('distance'))
        ls_enhanced_altitude.append(record.get_value('enhanced_altitude'))
        ls_enhanced_speed.append(record.get_value('enhanced_speed'))
        ls_heart_rate.append(record.get_value('heart_rate'))
        ls_position_lat.append(record.get_value('position_lat'))
        ls_position_long.append(record.get_value('position_long'))
        ls_speed.append(record.get_value('speed'))
        ls_timestamp.append(record.get_value('timestamp'))
        ls_vertical_speed.append(record.get_value('vertical_speed'))

        record_code += 1
    move_code += 1

In [72]:
# Creating DF with all the records from all the moves
records = pd.DataFrame({'idmove': ls_move_code,
                        'idrecord': ls_record_code,
                        'latitude': ls_position_lat,
                        'longitude': ls_position_long,
                        'distance': ls_distance,
                        'altitude': ls_altitude,
                        'enhanced_altitude': ls_enhanced_altitude,
                        'cadence': ls_cadence,
                        'speed': ls_speed,
                        'enhanced_speed': ls_enhanced_speed,
                        'vertical_speed': ls_vertical_speed,
                        'heart_rate': ls_heart_rate,
                        'timestamp': ls_timestamp 
                       })

#### Adding general move information to the table

In [69]:
list = ['lap', 'session', 'activity', 'file_id', 'file_creator']

for elem in fitfile.get_messages('lap'):
    print(elem)
    print(elem.get_values())
    print('_____________________________')
for elem in fitfile.get_messages('session'):
    print(elem)
    print(elem.get_values())
    print('_____________________________')
for elem in fitfile.get_messages('activity'):
    print(elem)
    print(elem.get_values())
    print('_____________________________')
for elem in fitfile.get_messages('file_id'):
    print(elem)
    print(elem.get_values())
    print('_____________________________')
for elem in fitfile.get_messages('file_creator'):
    print(elem)
    print(elem.get_values())

lap (#19)
{'start_time': datetime.datetime(2018, 6, 17, 6, 0, 24), 'timestamp': datetime.datetime(2018, 6, 17, 9, 38, 29), 'total_elapsed_time': 12924.834, 'total_timer_time': 12924.834, 'total_moving_time': 12924.834, 'total_distance': 24389.0, 'event': 'lap', 'lap_trigger': 'manual', 'event_type': 'stop', 'message_index': 0}
_____________________________
session (#18)
{'start_time': datetime.datetime(2018, 6, 17, 6, 0, 24), 'timestamp': datetime.datetime(2018, 6, 17, 9, 38, 29), 'sport': 'running', 'total_distance': 24389.0, 'total_elapsed_time': 12924.834, 'total_timer_time': 12924.834, 'total_moving_time': 12924.834, 'total_calories': 3409, 'message_index': 0, 'first_lap_index': 0, 'num_laps': 1, 'num_active_lengths': 0, 'event': 'session', 'event_type': 'stop'}
_____________________________
activity (#34)
{'timestamp': datetime.datetime(2018, 6, 17, 9, 38, 29), 'local_timestamp': datetime.datetime(2018, 6, 17, 11, 38, 29), 'total_timer_time': 0.0, 'num_sessions': 1, 'type': 'manua

In [ ]:
table_words.to_sql(con=engine, name='words', if_exists='replace')